In [38]:
import time
import numpy as np
from tqdm import tqdm

import json
import pandas as pd

from transformers import AutoTokenizer, AutoModel

import warnings
import os
import ast

warnings.filterwarnings('ignore')

import re
from unidecode import unidecode

from pymongo.mongo_client import MongoClient
import json

In [39]:
def compute_f1(prediction, truth):
    pred_tokens = prediction.split()
    truth_tokens = truth.split()
    
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [40]:
import string
def normalize(input_string):
    input_string = unidecode(input_string)
    input_string = input_string.lower()
        # Replace all punctuation with a space
    input_string = re.sub(f"[{re.escape(string.punctuation)}]", " ", input_string)
    
    # Replace multiple spaces with a single space
    input_string = re.sub(r"\s+", " ", input_string)
    
    # Trim leading/trailing whitespace
    return input_string.strip()

In [41]:
def get_mongo_client(mongo_uri):
    client = MongoClient(mongo_uri)
    return client

mongo_client = get_mongo_client("mongodb://localhost:27018/?directConnection=true")
# db = mongo_client.get_database("wikidata_ontology_test")
# db.list_collection_names()
mongo_client.list_database_names()

['admin',
 'config',
 'demo',
 'local',
 'non_onto_triplets_db_gpt4_1_mini',
 'non_onto_triplets_db_gpt4_o_mini',
 'onto_triplets_db_gpt4o-mini',
 'triplets_db',
 'triplets_db_gpt4_1_mini',
 'triplets_db_hotpot_gpt4_1_mini',
 'wikidata_ontology',
 'wikidata_ontology_test']

In [42]:
db = mongo_client.get_database("triplets_db_gpt4_1_mini")
ontology_db = mongo_client.get_database("wikidata_ontology")
# triplets_db = mongo_client.get_database("triplets_db_gpt4_1_mini")
triplets_db = mongo_client.get_database("triplets_db_gpt4_1_mini")

In [43]:
unique_sample_ids = triplets_db.get_collection('triplets').distinct('sample_id')
print(len(unique_sample_ids))


200


In [44]:
triplets_db.get_collection('triplets').find_one({})

{'_id': ObjectId('68cc7139d481a3884ea4a540'),
 'sample_id': '2hop__121145_561444',
 'subject': 'The Real L Word',
 'object': 'reality television series',
 'subject_type': 'television series',
 'object_type': 'genre',
 'relation': 'instance of',
 'completion_token_num': 20,
 'prompt_token_num': 497,
 'qualifiers': [],
 'source_text_id': 0}

In [45]:
from utils.openai_utils import LLMTripletExtractor
from utils.structured_aligner import Aligner
# from utils.dynamic_aligner import Aligner
from utils.structured_inference_with_db import StructuredInferenceWithDB
# from utils.inference_with_db import InferenceWithDB

model_name = 'gpt-4.1-mini'
aligner = Aligner(ontology_db=ontology_db, triplets_db=triplets_db)
# aligner = Aligner(triplets_db=triplets_db)

extractor = LLMTripletExtractor(model=model_name)
inferer = StructuredInferenceWithDB(extractor, aligner, triplets_db=triplets_db)
# inferer = InferenceWithDB(extractor, aligner, triplets_db=triplets_db)

with open("datasets/musique_200_test.json", "r") as f:
    ds = json.load(f)


In [46]:
# print(aligner.entities_vector_index_name)
# print(aligner.entity_aliases_collection_name)
# print(aligner.entity_type_vector_index_name)
# print(aligner.entity_type_aliases_collection_name)
# print(aligner.property_aliases_collection_name)
# print(aligner.property_collection_name)
# print(aligner.entity_type_collection_name)
# print(aligner.triplets_collection_name)
# print(aligner.filtered_triplets_collection_name)
# print(aligner.ontology_filtered_triplets_collection_name)
# print(aligner.initial_triplets_collection_name)


In [47]:
list(db.get_collection(aligner.triplets_collection_name).find({"sample_id": "2hop__121145_561444"}, {"_id": 0, "label": 1, "entity_type": 1, "sample_id": 1}))

[{'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121145_561444'},
 {'sample_id': '2hop__121

In [48]:
db.get_collection(aligner.entity_aliases_collection_name).find_one({})

{'_id': ObjectId('68cc7130910fed52b2fdfa58'),
 'label': 'The Real L Word',
 'entity_type': 'television series',
 'alias': 'The Real L Word',
 'sample_id': '2hop__121145_561444',
 'alias_text_embedding': [0.009889713488519192,
  -0.07133278995752335,
  -0.0059313117526471615,
  -0.030011042952537537,
  0.014623121358454227,
  -0.024449745193123817,
  0.037173010408878326,
  0.0037516832817345858,
  -0.09681671857833862,
  -0.015195912681519985,
  0.01731961779296398,
  -0.07978171855211258,
  -0.03222719952464104,
  -0.01421310007572174,
  -0.04445761814713478,
  -0.002834167331457138,
  -0.031816836446523666,
  -0.06295185536146164,
  -0.00504443421959877,
  0.03671497851610184,
  -0.00648869015276432,
  0.03426196053624153,
  -0.04647936299443245,
  -0.03071165643632412,
  -0.05182984843850136,
  0.08383699506521225,
  0.07503727823495865,
  0.035214636474847794,
  -0.03033829666674137,
  -0.06335499882698059,
  0.04653048887848854,
  0.0007498351042158902,
  -0.06709463149309158,
  -

In [49]:
with open("datasets/musique_200_test.json", "r") as f:
    ds = json.load(f)
    
ds = ds['data'][:]

id2sample = {}
for elem in ds:
    id2sample[elem['id']] = elem

unique_sample_ids = triplets_db.get_collection('triplets').distinct('sample_id')
print(len(unique_sample_ids))


200


In [50]:
unique_sample_ids

['2hop__10017_18974',
 '2hop__10122_18974',
 '2hop__10620_79092',
 '2hop__115515_779396',
 '2hop__121145_561444',
 '2hop__123148_5385',
 '2hop__128608_82341',
 '2hop__129721_40482',
 '2hop__13592_49388',
 '2hop__135993_160249',
 '2hop__14092_8311',
 '2hop__142443_768138',
 '2hop__142699_67465',
 '2hop__145082_35034',
 '2hop__145282_21711',
 '2hop__146919_29244',
 '2hop__149710_108549',
 '2hop__149862_108549',
 '2hop__16844_42173',
 '2hop__2148_78851',
 '2hop__24973_5674',
 '2hop__25344_88628',
 '2hop__25788_25839',
 '2hop__25788_990',
 '2hop__274110_413723',
 '2hop__276637_126089',
 '2hop__278022_71701',
 '2hop__28287_89399',
 '2hop__283_574195',
 '2hop__307218_161450',
 '2hop__335899_124498',
 '2hop__35136_35117',
 '2hop__35173_17335',
 '2hop__35209_17335',
 '2hop__358582_189042',
 '2hop__364489_861485',
 '2hop__365236_19320',
 '2hop__36747_18974',
 '2hop__3739_13529',
 '2hop__374495_68633',
 '2hop__374767_152023',
 '2hop__403060_92763',
 '2hop__403712_793210',
 '2hop__41158_22946',
 

In [51]:
sample_id2ans = {}
# ids = list(db.get_collection(aligner.triplets_collection_name).find({}, {"_id": 0, "sample_id": 1}))
# ids = [item["sample_id"] for item in ids]
# ids = set(ids)
# for sample_id in tqdm(ids):
# for sample_id in tqdm(id2sample.keys()):
for sample_id in tqdm(unique_sample_ids):
    try:
        question = id2sample[sample_id]['question']
        identified_entities = inferer.identify_relevant_entities_from_question(question=question, sample_id=sample_id)
        print(len(identified_entities))
        if not identified_entities:
            print(f"[DEBUG] No entities identified for sample_id={sample_id}, question={question}")
        # if len(identified_entities) > 0:
        supporting_triplets, ans = inferer.answer_question(question=question, relevant_entities=identified_entities, sample_id=sample_id, use_filtered_triplets=False)
        # supporting_triplets, ans = inferer.answer_question(question=question, relevant_entities=identified_entities, sample_id=sample_id)

        print(sample_id, " | ", question, ' | ', ans, " | ", normalize(ans), " | ", id2sample[sample_id]['answer'], " | ", normalize(id2sample[sample_id]['answer']))
        print(len(supporting_triplets))
        print("-"*100)
        sample_id2ans[sample_id] = ans
    except Exception as e:
        print(f"Error for sample_id={sample_id}, question={question}")
        continue

  0%|          | 0/200 [00:00<?, ?it/s]

Similar entities:  [{'entity': 'military titles'}, {'entity': 'military titles'}, {'entity': 'military detachment led by Lieutenant Colonel Richard I. Dodge'}, {'entity': 'army'}, {'entity': 'Nimrod Expedition'}, {'entity': 'Scott Expedition'}, {'entity': 'expedition from the ship Fram'}, {'entity': 'specialised military units and formal military occupations'}, {'entity': '7th-century Srivijaya siddhayatra expedition'}, {'entity': '7th-century Srivijaya siddhayatra expedition'}]
Similar entities:  [{'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'north Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Fujian province'}, {'entity': 'Zhejiang'}]
Similar entities:  [{'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'north Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Hangzhou'}, {

DEBUG:StructuredInferenceWithDB:Supporting triplets: [{'subject': 'Zhejiang', 'relation': 'instance of', 'object': 'province of China', 'qualifiers': []}, {'subject': 'eastern coastal region of China', 'relation': 'located in the administrative territorial entity', 'object': 'Zhejiang', 'qualifiers': []}, {'subject': 'Zhejiang', 'relation': 'named after', 'object': 'Chekiang', 'qualifiers': []}, {'subject': 'Zhejiang', 'relation': 'shares border with', 'object': 'Jiangsu province', 'qualifiers': [{'relation': 'direction', 'object': 'north'}]}, {'subject': 'Zhejiang', 'relation': 'shares border with', 'object': 'Shanghai municipality', 'qualifiers': [{'relation': 'direction', 'object': 'north'}]}, {'subject': 'Zhejiang', 'relation': 'shares border with', 'object': 'Anhui province', 'qualifiers': [{'relation': 'direction', 'object': 'northwest'}]}, {'subject': 'Zhejiang', 'relation': 'shares border with', 'object': 'Jiangxi province', 'qualifiers': [{'relation': 'direction', 'object': 'w

Chosen entities after identification:  [{'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Fujian province', 'entity_type': 'province'}]
8
Chosen relevant entities:  [{'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Zhejiang'}, {'entity': 'Fujian province', 'entity_type': 'province'}]


  0%|          | 1/200 [00:03<10:28,  3.16s/it]

...

Chosen entities after identification:  [{'entity': 'Derech Mitzvosecha'}, {'entity': 'Rabbi Dovber Schneuri', 'entity_type': 'third Rebbe'}]
2
Chosen relevant entities:  [{'entity': 'Derech Mitzvosecha'}, {'entity': 'Rabbi Dovber Schneuri', 'entity_type': 'third Rebbe'}]


  2%|▎         | 5/200 [00:13<08:53,  2.73s/it]

2hop__121145_561444  |  Who did the creator of Derech Mitzvosecha follow?  |  second Rebbe of the Chabad Hasidic movement  |  second rebbe of the chabad hasidic movement  |  Dovber Schneuri  |  dovber schneuri
9
----------------------------------------------------------------------------------------------------


  2%|▎         | 5/200 [00:14<09:12,  2.83s/it]


KeyboardInterrupt: 

In [33]:
len(sample_id2ans)

200

In [34]:
sample_ids = set()
for doc in db.get_collection(aligner.triplets_collection_name).find({}, {"_id": 0, "sample_id": 1}):
    sample_ids.add(doc["sample_id"])
len(sample_ids)

200

In [35]:
db.get_collection(aligner.entity_aliases_collection_name).find_one({"sample_id": "2hop__365236_19320"})

{'_id': ObjectId('68ce5d70cf8c45b8edc760eb'),
 'label': 'Harvard',
 'entity_type': 'university',
 'alias': 'Harvard',
 'sample_id': '2hop__365236_19320',
 'alias_text_embedding': [0.04436160996556282,
  0.015104020945727825,
  0.0010068441042676568,
  -0.0825362503528595,
  -0.01353803277015686,
  0.0003280105593148619,
  0.007344139274209738,
  0.015294904820621014,
  0.01312243565917015,
  -0.06016584113240242,
  0.006141675170511007,
  -0.0068809338845312595,
  -0.050348397344350815,
  -0.04428769275546074,
  -0.012345030903816223,
  -0.02315518818795681,
  -0.022969013080000877,
  -0.0586497038602829,
  -0.03450579568743706,
  0.02175183594226837,
  -0.016816848888993263,
  0.004256978631019592,
  0.009577223099768162,
  -0.07135123759508133,
  0.03629777207970619,
  -0.03210164234042168,
  -0.006687880959361792,
  0.018349772319197655,
  0.0156754981726408,
  0.04258941486477852,
  0.009269644506275654,
  0.0005786602268926799,
  -0.06181511655449867,
  -0.010824251919984818,
  0.

In [36]:
sample_id2ans

{'2hop__10017_18974': 'Lieutenant Colonel Richard I. Dodge',
 '2hop__10122_18974': '""',
 '2hop__10620_79092': 'Quaternary',
 '2hop__115515_779396': 'University of Glasgow',
 '2hop__121145_561444': 'second Rebbe of the Chabad Hasidic movement',
 '2hop__123148_5385': '11900',
 '2hop__128608_82341': 'Northern Florida',
 '2hop__129721_40482': 'British Landgrave Edmund Bellinger',
 '2hop__13592_49388': '38',
 '2hop__135993_160249': '""',
 '2hop__14092_8311': '""',
 '2hop__142443_768138': '',
 '2hop__142699_67465': 'March 11, 2011',
 '2hop__145082_35034': '1,306,300',
 '2hop__145282_21711': '15th century',
 '2hop__146919_29244': 'Paris-Charles de Gaulle Airport',
 '2hop__149710_108549': 'The team',
 '2hop__149862_108549': 'Jerry Siegel and Joe Shuster',
 '2hop__16844_42173': '1757',
 '2hop__2148_78851': 'Brooklyn Nets',
 '2hop__24973_5674': 'President Yeltsin',
 '2hop__25344_88628': 'John Wayne played U.S. Marshal Rooster Cogburn in True Grit.',
 '2hop__25788_25839': '900 mA',
 '2hop__25788

In [37]:
f1s = []
ems = []
for sample_id in id2sample.keys():
# for sample_id in ids:
    if sample_id in sample_id2ans:
        question = id2sample[sample_id]['question']
        ans = sample_id2ans[sample_id]
        max_f1 = 0
        max_em = 0
        max_f1_entity = ''
        max_em_entity = ''
        gold_answers_variations = [id2sample[sample_id]['answer']]
        gold_answers_variations.extend(id2sample[sample_id]['answer_aliases'])
        for golden_answer in gold_answers_variations:
            golden_answer = normalize(golden_answer)
            ans = normalize(ans)
            f1 = compute_f1(prediction=ans, truth=golden_answer)
            em = golden_answer == ans


            if f1 > max_f1:
                max_f1_entity = golden_answer
            if em > max_em:
                max_em_entity = golden_answer
            max_f1 = max(max_f1, f1)
            max_em = max(max_em, em)
        
        print(sample_id, " | ", ans, " | ", max_em_entity, " | ", max_f1_entity)
        f1s.append(max_f1)
        ems.append(max_em)

sum(ems) / len(ems), sum(f1s)/len(f1s)

2hop__121145_561444  |  second rebbe of the chabad hasidic movement  |    |  
2hop__86689_728109  |  los angeles lakers  |    |  
3hop1__462960_160545_62931  |  koh phi phi  |    |  island koh phi phi
3hop1__68732_39743_24526  |  upper 40s lower 50s degf 8 12 degc during the day  |    |  upper 40s lower 50s degf
2hop__364489_861485  |  the triplets do not specify the headquarters of minnesota historical society press the publisher of bamboo among the oaks  |    |  minnesota history center
2hop__835710_7298  |  josh groban  |    |  
2hop__96062_159673  |  escape from new york  |    |  
3hop2__79512_16214_84681  |  17th century  |    |  
3hop1__831499_228453_10972  |  1970s  |  1970s  |  1970s
3hop2__230_89048_66294  |  john ritter  |    |  
2hop__96414_47902  |  damon  |    |  matt damon
3hop1__462960_160545_34754  |  having been occupied for over 30 years  |    |  
2hop__142699_67465  |  march 11 2011  |  march 11 2011  |  march 11 2011
2hop__115515_779396  |  university of glasgow  | 

(0.265, 0.39067147741466235)

In [63]:
accs_onto_gpt41_mini = [(0.31, 0.42185180894401286), (0.3, 0.4182425486391004), (0.3, 0.4207100811066329)]

In [64]:
print("EM: ", np.mean([elem[0] for elem in accs_onto_gpt41_mini]), np.std([elem[0] for elem in accs_onto_gpt41_mini]))
print("F1: ", np.mean([elem[1] for elem in accs_onto_gpt41_mini]), np.std([elem[1] for elem in accs_onto_gpt41_mini]))

EM:  0.3033333333333333 0.004714045207910321
F1:  0.4202681462299154 0.001506246950447532


In [12]:
with open("datasets/hotpotqa200.json", "r") as f:
    ds = json.load(f)

ds = ds[:50]
id2sample = {}
for elem in ds:
    id2sample[elem['_id']] = elem

In [13]:
sample_id2ans = {}
# ids = list(db.get_collection(aligner.triplets_collection_name).find({}, {"_id": 0, "sample_id": 1}))
# ids = [item["sample_id"] for item in ids]
# ids = set(ids)
# for sample_id in tqdm(ids):
for sample_id in tqdm(id2sample.keys()):
    # try:
        question = id2sample[sample_id]['question']
        identified_entities = inferer.identify_relevant_entities_from_question(question=question, sample_id=sample_id)
        print(len(identified_entities))
        if not identified_entities:
            print(f"[DEBUG] No entities identified for sample_id={sample_id}, question={question}")
        # if len(identified_entities) > 0:
        supporting_triplets, ans = inferer.answer_question(question=question, relevant_entities=identified_entities, sample_id=sample_id)

        print(sample_id, " | ", question, ' | ', ans, " | ", normalize(ans), " | ", id2sample[sample_id]['answer'], " | ", normalize(id2sample[sample_id]['answer']))
        print(len(supporting_triplets))
        print("-"*100)
        sample_id2ans[sample_id] = ans
    # except Exception as e:
    #     print(f"Error for sample_id={sample_id}, question={question}")
    #     continue

  0%|          | 0/50 [00:00<?, ?it/s]

Similar entities:  [{'entity': 'VIVA Media AG', 'entity_type': 'organization'}, {'entity': 'VIVA Media', 'entity_type': 'company'}, {'entity': 'VIVA Media GmbH', 'entity_type': 'organization'}, {'entity': 'Viva', 'entity_type': 'television channel'}, {'entity': 'Viva', 'entity_type': 'television channel'}, {'entity': 'VIVA Polska', 'entity_type': 'television channel'}, {'entity': 'ProSiebenSat.1 Media AG', 'entity_type': 'company'}, {'entity': 'EM.TV & Merchandising AG', 'entity_type': 'company'}, {'entity': 'em.sport media ag', 'entity_type': 'company'}, {'entity': 'VIVA Polska', 'entity_type': 'television channel'}]
Identified entities:  []
Chosen entities:  [{'entity': 'VIVA Media AG', 'entity_type': 'organization'}]


DEBUG:StructuredInferenceWithDB:Supporting triplets: [{'subject': 'Constantin Medien AG', 'relation': 'beforehand owned by', 'object': 'EM.Entertainment', 'qualifiers': []}, {'subject': 'Constantin Medien AG', 'relation': 'different from', 'object': 'EM.TV & Merchandising AG', 'qualifiers': []}, {'subject': 'Constantin Medien AG', 'relation': 'follows', 'object': 'EM.TV & Merchandising AG', 'qualifiers': []}, {'subject': 'Constantin Medien AG', 'relation': 'merged into', 'object': 'em.sport media ag', 'qualifiers': []}, {'subject': 'Constantin Medien AG', 'relation': 'country of origin', 'object': 'Germany', 'qualifiers': []}, {'subject': 'Constantin Medien AG', 'relation': 'location', 'object': 'Ismaning', 'qualifiers': [{'relation': 'near', 'object': 'Munich'}]}, {'subject': 'Constantin Medien AG', 'relation': 'industry', 'object': 'media', 'qualifiers': []}, {'subject': 'Constantin Medien AG', 'relation': 'field of work', 'object': 'sports marketing', 'qualifiers': []}, {'subject': 

Chosen entities after identification:  [{'entity': 'VIVA Media AG', 'entity_type': 'organization'}]
1
Chosen relevant entities:  [{'entity': 'VIVA Media AG', 'entity_type': 'organization'}]


  2%|▏         | 1/50 [00:02<01:57,  2.39s/it]

...

5a72dcb45542992359bc31af  |  What company did Rex Maughan aquire?  |  Forever Living Products International, Inc.  |  forever living products international inc  |  Aloe Vera of America  |  aloe vera of america
120
----------------------------------------------------------------------------------------------------
Similar entities:  [{'entity': 'Claudette', 'entity_type': 'character (fictional human or non-human character in a narrative work of art)'}, {'entity': 'Brenda', 'entity_type': 'television character'}, {'entity': 'Marge Simpson', 'entity_type': 'television character'}, {'entity': 'Canada', 'entity_type': 'country'}, {'entity': 'Tricia Helfer', 'entity_type': 'human'}, {'entity': 'Harvey', 'entity_type': 'television character'}, {'entity': 'Canadian', 'entity_type': 'nationality'}, {'entity': 'Tricia Helfer', 'entity_type': 'human'}, {'entity': 'actress', 'entity_type': 'profession'}, {'entity': 'Patty Bouvier', 'entity_type': 'television character'}]
Similar entities:  [{'enti

DEBUG:StructuredInferenceWithDB:Supporting triplets: [{'subject': 'Max Charles', 'relation': 'date of birth', 'object': 'August 18, 2003', 'qualifiers': []}, {'subject': 'Max Charles', 'relation': 'country of citizenship', 'object': 'American', 'qualifiers': []}, {'subject': 'Max Charles', 'relation': 'occupation', 'object': 'teen actor', 'qualifiers': []}, {'subject': 'Max Charles', 'relation': 'subject has role', 'object': 'young Peter Parker', 'qualifiers': [{'relation': 'in work', 'object': 'The Amazing Spider-Man'}]}, {'subject': 'The Neighbors', 'relation': 'performer', 'object': 'Max Charles', 'qualifiers': [{'relation': 'genre', 'object': 'comedy science fiction series'}, {'relation': 'network', 'object': 'ABC'}]}, {'subject': 'Sherman', 'relation': 'voice actor', 'object': 'Max Charles', 'qualifiers': [{'relation': 'in work', 'object': 'Mr. Peabody & Sherman'}, {'relation': 'production company', 'object': 'DreamWorks Animation'}, {'relation': 'point in time', 'object': '2014'}

Chosen entities after identification:  [{'entity': 'Walk All Over Me', 'entity_type': 'film'}, {'entity': 'Tricia Helfer', 'entity_type': 'human'}, {'entity': 'Starcraft 2', 'entity_type': 'video game'}]
3
Chosen relevant entities:  [{'entity': 'Walk All Over Me', 'entity_type': 'film'}, {'entity': 'Tricia Helfer', 'entity_type': 'human'}, {'entity': 'Starcraft 2', 'entity_type': 'video game'}]


100%|██████████| 50/50 [02:25<00:00,  2.92s/it]

5ae1e22a5542997f29b3c160  |  The role of "Celene" in the film "Walk All over Me" was played by an actress that voices what role in the "Starcraft 2" triolgy?  |  Tricia Helfer voices Sarah Kerrigan (Queen of Blades) in the Starcraft 2 trilogy.  |  tricia helfer voices sarah kerrigan queen of blades in the starcraft 2 trilogy  |  Sarah Kerrigan  |  sarah kerrigan
122
----------------------------------------------------------------------------------------------------


In [14]:
f1s = []
ems = []
for sample_id in id2sample.keys():
        question = id2sample[sample_id]['question']
        ans = sample_id2ans[sample_id]
        max_f1 = 0
        max_em = 0
        max_f1_entity = ''
        max_em_entity = ''
        gold_answers_variations = [id2sample[sample_id]['answer']]
        # gold_answers_variations.extend(id2sample[sample_id]['answer_aliases'])
        for golden_answer in gold_answers_variations:
            golden_answer = normalize(golden_answer)
            ans = normalize(ans)
            f1 = compute_f1(prediction=ans, truth=golden_answer)
            em = golden_answer == ans


            if f1 > max_f1:
                max_f1_entity = golden_answer
            if em > max_em:
                max_em_entity = golden_answer
            max_f1 = max(max_f1, f1)
            max_em = max(max_em, em)
        
        print(sample_id, " | ", ans, " | ", max_em_entity, " | ", max_f1_entity)
        f1s.append(max_f1)
        ems.append(max_em)

sum(ems) / len(ems), sum(f1s)/len(f1s)

5a7613c15542994ccc9186bf  |  music television network  |    |  
5adf2fa35542993344016c11  |  jonny craig  |  jonny craig  |  jonny craig
5adfdef9554299025d62a36b  |    |    |  
5a7180205542994082a3e856  |  creature comforts  |  creature comforts  |  creature comforts
5a78bc6b554299148911f979  |  magazines of women interest  |    |  fortnightly women interest magazine
5abdd0f15542991f6610604d  |  failed coup attempt  |    |  a failed coup attempt
5a8e27d45542995a26add46a  |  2004  |  2004  |  2004
5a881d2355429938390d3eeb  |  love and theft  |  love and theft  |  love and theft
5ae4a1ef55429970de88d9e7  |  2 march 1972  |  2 march 1972  |  2 march 1972
5ae6f2a7554299572ea5464a  |  romeo montague  |    |  
5a845d735542996488c2e52e  |  virginia  |  virginia  |  virginia
5adf5daf5542995534e8c79d  |  darren benjamin shepherd is american remi lange is french  |    |  
5a7c49dc55429935c91b514f  |  science  |    |  
5abe364e5542993f32c2a08e  |  teen titans go  |  teen titans go  |  teen titans

(0.52, 0.6709603174603174)

In [ ]:
0.52, 0.6709603174603174

(0.48, 0.6520100250626567)

In [24]:
np.mean([0.6509191159717475, 0.6520100250626567, 0.6642524493050809])

0.6557271967798283

In [25]:
np.std([0.6509191159717475, 0.6520100250626567, 0.6642524493050809])

0.006044692913382812